# Calculating lsr climo
### CWA

In [1]:
import datetime
import glob
import json
import numpy as np
import pandas as pd
import math

In [12]:
wind_files = glob.glob('../lsrs/201*-wind*')
hail_files = glob.glob('../lsrs/201*-hail*')

In [3]:
# for the time being, a list of the CWAs we're investigating
cwas = ['FWD','OUN','TSA','SHV','HGX','EWX','SJT']

In [78]:
# Set up a dict to hold data for cwas
def makeHolder():
    statHolder = {}
    for cwa in cwas:
        statHolder[cwa] = {'hail': np.zeros(366), 'hailDays': np.zeros(366),'wind': np.zeros(366),'windDays': np.zeros(366)}
    return statHolder

Prep dataframe

In [73]:
def dfPrep(df):

    df['dayStr'] = df[['year','month','day']].astype(str).agg('-'.join,axis=1)
    df['dayIdx'] = pd.to_datetime(df['dayStr']).dt.dayofyear
    df['dayIdx'] = df['dayIdx'].apply(leapAdjust)

    df['CWA'] = df['comments'].apply(lambda x: x[x.find("(")+1:x.find(")")])
    
    return df

In [83]:
statHolder = makeHolder()

for file in wind_files:
    print(f'Working on file {file}')
    df = pd.read_csv(file,sep='|')
    
    # Get correct dayIdx and CWA formatting
    df = dfPrep(df)
    
    # Filter by CWA
    for cwa in cwas:
        print(f'CWA: {cwa}')
        df_cwa = df[df['CWA'] == cwa]
        
        for i, row in df_cwa.iterrows():
            
            # Add report totals to dict
            statHolder[cwa]['wind'][row['dayIdx']-1] += 1
            
        # Get count of report days in the dict
        windDays = df_cwa.groupby('dayIdx').count().index
        for day in windDays:
            statHolder[cwa]['windDays'][day-1] += 1
            
print('Now it is the time for the hail!')            

for file in hail_files:
    print(f'Working on file {file}')
    df = pd.read_csv(file,sep='|')
    
    # Get correct dayIdx and CWA formatting
    df = dfPrep(df)
    
    # Filter by CWA
    for cwa in cwas:
        print(f'CWA: {cwa}')
        df_cwa = df[df['CWA'] == cwa]
        
        for i, row in df_cwa.iterrows():
            
            # Add report totals to dict
            statHolder[cwa]['hail'][row['dayIdx']-1] += 1
            
        # Get count of report days in the dict
        hailDays = df_cwa.groupby('dayIdx').count().index
        for day in hailDays:
            statHolder[cwa]['hailDays'][day-1] += 1

Working on file ../lsrs/2012-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2017-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2018-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2016-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2013-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2019-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2010-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2015-wind-prelim-reports-ytd.csv
CWA: FWD
CWA: OUN
CWA: TSA
CWA: SHV
CWA: HGX
CWA: EWX
CWA: SJT
Working on file ../lsrs/2014-wind-prelim

In [84]:
for cwa in statHolder.keys():
    for type in statHolder[cwa].keys():
        statHolder[cwa][type] = statHolder[cwa][type].tolist()

with open('lsr_climo_raw.json','w') as outfile:
    json.dump(statHolder,outfile)

Function to adjust dayofyear for leap years

In [38]:
def leapAdjust(x):
    if x > 58:
        return x + 1
    
    return x

#### Smooth and average

In [85]:
# This function adds values to the beginning and end of an array to make it ready for summing
def arrExtender(array,window):
    halfwin = math.floor(window/2)
    
    rando = array
    rando = np.array(rando)
    rando1 = np.insert(rando,0,rando[-halfwin:])
    rando2 = np.append(rando1,rando[:halfwin])
    
    return rando2

In [86]:
def summer(array,window):

    total = np.array([])

    start = 0
    end = window
    while end < (len(array)+1):
        total = np.append(total,np.sum(array[start:end]))
        start += 1
        end += 1
        
    return total

In [97]:
window = 91

for cwa in cwas:
    hailDays = summer(arrExtender(statHolder[cwa]['hailDays'],window),window)
    hailReps = summer(arrExtender(statHolder[cwa]['hail'],window),window)
    
    statHolder[cwa]['movAvg_hail'] = (hailReps/hailDays).tolist()
    
    windDays = summer(arrExtender(statHolder[cwa]['windDays'],window),window)
    windReps = summer(arrExtender(statHolder[cwa]['wind'],window),window)
    
    statHolder[cwa]['movAvg_wind'] = (windReps/windDays).tolist()

In [103]:
with open('lsr_climo_smAvg.json','w') as outfile:
    json.dump(statHolder,outfile)

In [110]:
np.sum(statHolder['FWD']['hail'][270:300]),np.sum(statHolder['FWD']['hailDays'][270:300])

(98.0, 20.0)

In [113]:
statHolder['FWD']['movAvg_wind'][284]

4.9743589743589745